In [1]:
## Construct LSTM using Tensorflow + Keras
# Import Libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras import optimizers

#Import the helper functions
#from helperFunctions import CSP
from helperFunctions import GetCombinedData_HaLT as GetHALT
from helperFunctions import GetCombinedData_CLA as GetCLA
from helperFunctions import GetCombinedData_5F as Get5F
from helperFunctions import GetCombinedData_FreeForm as GetFree

print("Available paradigms:\nHand, Leg, Tongue (HALT)\nClassic - L/R Hand (CLA)\nFingers (5F)")
paradigm = input('Enter desired paradigm: ');
directoryPath = input('Enter the relative path to data: ');
if(paradigm == 'HALT'):
    Data, Targets, DataCSP, TargetsCSP = GetHALT(directoryPath, True);
    numOutputs = 5;
elif(paradigm == 'CLA'):
    Data, Targets, DataCSP, TargetsCSP = GetCLA(directoryPath, True);
    print("############")
    print("Processing FreeForm data for further testing")
    FreeData, FreeTargets, FreeDataCSP, FreeTargetsCSP = GetFree(directoryPath, True);
    numOutputs = 2;
elif(paradigm == '5F'):
    Data, Targets, DataCSP, TargetsCSP = Get5F(directoryPath, True);
    numOutputs = 5;
else:
    print("Error: Invalid paradigm {}".format(paradigm));
    quit()

enableDropout = True;
dropoutPercentage = 0.3;

#Run GetCombinedData to pull the datasets from multiple subjects into a single set
#Data, Targets, DataCSP, TargetsCSP = GetHALT('../../../matDown/HaLT_Data', True);

## Split into train and test sets
DataTrain, DataTest, TargetsTrain, TargetsTest = train_test_split(Data, Targets, test_size=0.3, random_state=0)

## Reshape the data for time-series processing
## Syntax np.reshape((numExamples, numTimeSteps, numInputs/numFeatures))
DataTrainRe = DataTrain.reshape((DataTrain.shape[0], DataTrain.shape[1], DataTrain.shape[2]))
DataTestRe = DataTest.reshape((DataTest.shape[0], DataTest.shape[1], DataTest.shape[2]))

Available paradigms:
Hand, Leg, Tongue (HALT)
Classic - L/R Hand (CLA)
Fingers (5F)
Enter desired paradigm: CLA
Enter the relative path to data: ../../../matDown
Processing dataset 1 of 14
Processing dataset 2 of 14
Processing dataset 3 of 14
Processing dataset 4 of 14
Processing dataset 5 of 14
Processing dataset 6 of 14
Processing dataset 7 of 14
Processing dataset 8 of 14
Processing dataset 9 of 14
Processing dataset 10 of 14
Processing dataset 11 of 14
Processing dataset 12 of 14
Processing dataset 13 of 14
Processing dataset 14 of 14


/Users/daniel/Documents/GitHub/ECE228ECGProject/JupyterNotebooks/helperFunctions.py:83: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


############
Processing FreeForm data for further testing
Processing dataset 1 of 3
Processing dataset 2 of 3
Processing dataset 3 of 3


## Building  and training the model
After processing the data, we build the LSTM model.
We use 100 units in the first LSTM layer, 50 units in the second LSTM layer, and 2 or 5 dense units in the final layer (depending on the paradigm).

We first train the network on non-CSP data.

In [2]:
LSTM_EEG = Sequential()
LSTM_EEG.add(LSTM((100),batch_input_shape=(None,DataTrainRe.shape[1], DataTrainRe.shape[2]), return_sequences=True))
if(enableDropout):
    LSTM_EEG.add(Dropout(dropoutPercentage))
LSTM_EEG.add(LSTM((50), return_sequences=False))
if(enableDropout):
    LSTM_EEG.add(Dropout(dropoutPercentage))
LSTM_EEG.add(Dense((numOutputs),activation='sigmoid'))

LSTM_EEG.summary()
sgd = optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
LSTM_EEG.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

history = LSTM_EEG.fit(DataTrain, TargetsTrain, epochs=30,verbose=2, batch_size=16)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 21, 100)           120800    
_________________________________________________________________
dropout (Dropout)            (None, 21, 100)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 2)                 102       
Total params: 151,102
Trainable params: 151,102
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
370/370 - 5s - loss: 0.6982 - accuracy: 0.5173
Epoch 2/30
370/370 - 5s - loss: 0.6848 - accuracy: 0.5653


## Evaluating the model
Now, we use the model to predict the outputs for the test set, and compare them to the target outputs.

In [3]:
predictionsTest = LSTM_EEG.predict(DataTest)

predictionsTest[predictionsTest>0.5] = 1
predictionsTest[predictionsTest <= 0.5] = 0
comparisonArrayTest = predictionsTest == TargetsTest

correctCountTest = 0
for boolValues in comparisonArrayTest:
    if(boolValues[0] & boolValues[1]):
        correctCountTest += 1
falseCountTest = DataTest.shape[0] - correctCountTest

predictionsTrain = LSTM_EEG.predict(DataTrain)
predictionsTrain[predictionsTrain>0.5] = 1;
predictionsTrain[predictionsTrain<=0.5] = 0;
comparisonArrayTrain = predictionsTrain == TargetsTrain;

correctCountTrain = 0
for boolValues in comparisonArrayTrain:
    if(boolValues[0] & boolValues[1]):
        correctCountTrain += 1
falseCountTrain = DataTrain.shape[0] - correctCountTrain


# In[87]:
trainAcc_noCSP = (correctCountTrain*100/DataTrain.shape[0]);
testAcc_noCSP = (correctCountTest*100/DataTest.shape[0]);

print("#################################")
print("#################################")
print("Training Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTrain, falseCountTrain, trainAcc_noCSP))
print("#################################")
print("#################################")
print("Testing Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTest, falseCountTest, testAcc_noCSP))
print("#################################")
print("#################################")

#################################
#################################
Training Performance:
Correct MI Prediction: 4837
Incorrect MI Prediction: 1082
Percent Accuracy: 81.720%
#################################
#################################
Testing Performance:
Correct MI Prediction: 1490
Incorrect MI Prediction: 1047
Percent Accuracy: 58.731%
#################################
#################################


## Using CSP Data
Now, we train the model using the CSP data and then evaluate it

In [4]:
DataTrain, DataTest, TargetsTrain, TargetsTest = train_test_split(DataCSP, TargetsCSP, test_size=0.3, random_state=0)

## Reshape the data for time-series processing
## Syntax np.reshape((numExamples, numTimeSteps, numInputs/numFeatures))
DataTrainRe = DataTrain.reshape((DataTrain.shape[0], DataTrain.shape[1], DataTrain.shape[2]))
DataTestRe = DataTest.reshape((DataTest.shape[0], DataTest.shape[1], DataTest.shape[2]))

history = LSTM_EEG.fit(DataTrain, TargetsTrain, epochs=30,verbose=2, batch_size=16)

predictionsTest = LSTM_EEG.predict(DataTest)

predictionsTest[predictionsTest>0.5] = 1
predictionsTest[predictionsTest <= 0.5] = 0
comparisonArrayTest = predictionsTest == TargetsTest

correctCountTest = 0
for boolValues in comparisonArrayTest:
    if(boolValues[0] & boolValues[1]):
        correctCountTest += 1
falseCountTest = DataTest.shape[0] - correctCountTest

predictionsTrain = LSTM_EEG.predict(DataTrain)
predictionsTrain[predictionsTrain>0.5] = 1;
predictionsTrain[predictionsTrain<=0.5] = 0;
comparisonArrayTrain = predictionsTrain == TargetsTrain;

correctCountTrain = 0
for boolValues in comparisonArrayTrain:
    if(boolValues[0] & boolValues[1]):
        correctCountTrain += 1
falseCountTrain = DataTrain.shape[0] - correctCountTrain


# In[87]:
trainAcc_noCSP = (correctCountTrain*100/DataTrain.shape[0]);
testAcc_noCSP = (correctCountTest*100/DataTest.shape[0]);

print("#################################")
print("#################################")
print("Training Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTrain, falseCountTrain, trainAcc_noCSP))
print("#################################")
print("#################################")
print("Testing Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTest, falseCountTest, testAcc_noCSP))
print("#################################")
print("#################################")

Epoch 1/30
370/370 - 6s - loss: 0.6823 - accuracy: 0.5898
Epoch 2/30
370/370 - 6s - loss: 0.5705 - accuracy: 0.7021
Epoch 3/30
370/370 - 6s - loss: 0.5072 - accuracy: 0.7452
Epoch 4/30
370/370 - 6s - loss: 0.4807 - accuracy: 0.7618
Epoch 5/30
370/370 - 6s - loss: 0.4586 - accuracy: 0.7794
Epoch 6/30
370/370 - 6s - loss: 0.4394 - accuracy: 0.7937
Epoch 7/30
370/370 - 6s - loss: 0.4361 - accuracy: 0.7946
Epoch 8/30
370/370 - 6s - loss: 0.4135 - accuracy: 0.8088
Epoch 9/30
370/370 - 6s - loss: 0.3904 - accuracy: 0.8196
Epoch 10/30
370/370 - 6s - loss: 0.3850 - accuracy: 0.8219
Epoch 11/30
370/370 - 7s - loss: 0.3831 - accuracy: 0.8251
Epoch 12/30
370/370 - 6s - loss: 0.3738 - accuracy: 0.8268
Epoch 13/30
370/370 - 6s - loss: 0.3530 - accuracy: 0.8419
Epoch 14/30
370/370 - 6s - loss: 0.3435 - accuracy: 0.8436
Epoch 15/30
370/370 - 6s - loss: 0.3385 - accuracy: 0.8525
Epoch 16/30
370/370 - 6s - loss: 0.3154 - accuracy: 0.8625
Epoch 17/30
370/370 - 6s - loss: 0.3002 - accuracy: 0.8719
Epoch 

## Optional testing of small FreeForm Datasets
The FREEFORM datasets are a collection of datasets (3 of them) that are self-paced. There may not be enough data to train the network on just the FreeForm dataset. If you do train it on that dataset, you might have issues with overfitting. So, if the paradigm was CLA, which is similar to FreeForm, we can evaluate the model on FreeForm as well.

In [9]:
if(paradigm == 'CLA'):
    ## Have to check and make sure that the FreeData is the same dimensions as the CLA data
    lenFree = FreeDataCSP.shape[2];
    if(lenFree < len(DataCSP)):
        FreeDataCSP = FreeDataCSP[:, :, 0:DataCSP.shape[2]];
        
    predictionsFree = LSTM_EEG.predict(FreeDataCSP)
    predictionsFree[predictionsFree>0.5] = 1;
    predictionsFree[predictionsFree<=0.5] = 0;
    comparisonArray = predictionsFree == FreeTargetsCSP;

    correctCount = 0
    for boolValues in comparisonArray:
        if(boolValues[0] & boolValues[1]):
            correctCount += 1
    falseCount = FreeDataCSP.shape[0] - correctCount


    # In[87]:
    Accuracy = (correctCount*100/FreeDataCSP.shape[0]);

    print("#################################")
    print("#################################")
    print("FreeForm Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCount, falseCount, Accuracy))
    print("#################################")
    print("#################################")

#################################
#################################
FreeForm Performance:
Correct MI Prediction: 1426
Incorrect MI Prediction: 700
Percent Accuracy: 67.074%
#################################
#################################


(2126, 21, 204)

2126